In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import requests
import time
import numpy as np

import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import alphavantage as av

In [2]:
## set environment variables
#os.environ["APIKEY"] = "*************"
#print(os.getenv("APIKEY"))

## Thrifty-Rabbit Project
### Project Goals

Goals:
1. Monitor and evaluate a dividend portfolio.
2. Create an optimized value portfolio.



#### Data Import and Processing

Step one is to have a way to get stock prices and fundamental data needed to evaulate an asset into our program. 

Alpha-vantage offers a free api that will allow me to get all the data I need to analyze an asset. 

I will use three different endpoints to collect the data needed to analyze the asset:


In [3]:
#Build portfolio
portfolio = [
    "SPY",
    "WMT",
    "JNPR",
    "CVS",
    "BAC",
    "JNJ",
    "ABB",
    "ORCL",
] 

In [4]:
# get the monthly adjusted prices for the portfolio symbols using AlphaVantage API
prices = pd.DataFrame()
for symbol in portfolio:
    df = av.get_monthly_adj_ts(symbol=symbol)
    df["symbol"] = symbol
    prices = pd.concat([prices,df])
    time.sleep(10) #sleep to avoid going over 5 calls per minute limit
prices.sort_values('DATE')
    

In [9]:
prices.head()

,DATE,OPEN,HIGH,LOW,CLOSE,ADJ_CLOSE,VOLUME,YIELD,symbol
280,1999-12-31,139.3125,147.5625,139.0000,146.8750,95.8116,121529300,0.3476,SPY
279,2000-01-31,148.2500,148.2500,135.0000,139.5625,91.0414,156770800,0.0000,SPY
278,2000-02-29,139.7500,144.5625,132.7187,137.4375,89.6552,186938300,0.0000,SPY
277,2000-03-31,137.6250,155.7500,135.0312,150.3750,98.3423,247594900,0.3708,SPY
276,2000-04-28,150.1250,153.1093,133.5000,145.0937,94.8885,229246200,0.0000,SPY


## Analysis

In [8]:
fig = px.line(prices, x="DATE", y="ADJ_CLOSE", title='My Portfolio', color='symbol')
fig.update_layout(xaxis=dict(range=["2020-01-01", "2023-03-31"]))
fig.show()

### Fundamentals

Let's get the company info so we can see what the fundamentals look like.

I'll be looking at:
- Company Overview
- Balance Sheet
- Cash Flow
- Income Statement

Let's do this just for one company to get an idea of what our data model will look like. 

Bank of America (BAC) seems like a good pick.